# Rerank 性能与超时调试
该 Notebook 用于排查重排序（Rerank）接口在大数据量下超时的原因。我们将分步测试不同 `top_k` 值下的延迟。

In [1]:
import requests
import json
import time

API_BASE_URL = "http://localhost:8000"
QUERY = "DPF_OUTP_HIGH_ERROR有哪些标定参数？"

print(f"API 地址: {API_BASE_URL}")
print(f"测试查询词: {QUERY}")

API 地址: http://localhost:8000
测试查询词: DPF_OUTP_HIGH_ERROR有哪些标定参数？


In [2]:
def call_search(query, top_k=5):
    url = f"{API_BASE_URL}/search"
    payload = {
        "query": query,
        "top_k": top_k,
        "strategy": "global"
    }
    start = time.time()
    response = requests.post(url, json=payload)
    latency = (time.time() - start) * 1000
    return response.json(), latency

def call_rerank(query, documents_str):
    url = f"{API_BASE_URL}/rerank"
    payload = {
        "model": "bge-reranker-v2-m3",
        "query": query,
        "documents": documents_str
    }
    start = time.time()
    # 增加超时时间到 300s，确保在后端处理大数据量时不被客户端查断
    try:
        response = requests.post(url, json=payload, timeout=300)
        latency = (time.time() - start) * 1000
        return response.json(), latency
    except requests.exceptions.Timeout:
        return {"error": "Timeout after 300s"}, (time.time() - start) * 1000
    except Exception as e:
        return {"error": str(e)}, (time.time() - start) * 1000

### 基础测试 (top_k=5)
验证逻辑是否跑通。

In [6]:
search_res, s_lat = call_search(QUERY, top_k=5)
print(f"搜索延迟: {s_lat:.2f} ms")

search_body_str = json.dumps(search_res)
rerank_res, r_lat = call_rerank(QUERY, search_body_str)

if "error" in rerank_res:
    print(f"❌ 重排序失败: {rerank_res['error']}")
else:
    print(f"✅ 重排序成功 | 延迟: {r_lat:.2f} ms | 结果数: {len(rerank_res.get('pure_documents', []))}")

搜索延迟: 626.18 ms
✅ 重排序成功 | 延迟: 21820.11 ms | 结果数: 5


### 压力测试（分级数据量测试）
通过不同数量的 `top_k` 来观察延迟的变化情况，定位瓶颈。

In [ ]:
test_ks = [30, 60]
results = []

for k in test_ks:
    print(f"\n--- 测试 top_k = {k} ---")
    
    # 搜索
    s_res, s_lat = call_search(QUERY, top_k=k)
    s_docs = s_res.get("pure_documents", [])
    
    # 序列化
    s_body_str = json.dumps(s_res)
    payload_size_kb = len(s_body_str.encode('utf-8')) / 1024
    
    # 重排序
    # 注意：现在后端 api.py 中已加入 batch_size=10 的逻辑
    r_res, r_lat = call_rerank(QUERY, s_body_str)
    
    status = "✅ 成功" if "error" not in r_res else f"❌ 失败 ({r_res['error']})"
    
    print(f"数据量: {len(s_docs)} 条 | Payload: {payload_size_kb:.2f} KB")
    print(f"搜索耗时: {s_lat/1000:.2f} s | 重排序耗时: {r_lat/1000:.2f} s")
    print(f"状态: {status}")
    
    results.append({
        "k": k,
        "search_s": s_lat/1000,
        "rerank_s": r_lat/1000,
        "payload_kb": payload_size_kb,
        "success": "error" not in r_res
    })

print("\n\n测试完成！")


--- 测试 top_k = 10 ---
数据量: 10 条 | Payload: 1.75 KB
搜索耗时: 0.43 s | 重排序耗时: 15.40 s
状态: ✅ 成功

--- 测试 top_k = 30 ---
数据量: 30 条 | Payload: 18.89 KB
搜索耗时: 0.50 s | 重排序耗时: 300.02 s
状态: ❌ 失败 (Timeout after 300s)

--- 测试 top_k = 60 ---


### 总结与分析
如果重排序在 30-45 条左右开始超时（默认 60s），说明是模型处理能力（CPU 瓶颈）导致。

目前已将后端 `api.py` 中的超时时间从 60s 增加到 300s，并将本 Notebook 中的请求超时也增加到 300s。这应该能解决处理较多文档（如 60 条以上）时的超时问题。